# Practice Session 03: Management of networks data
<font size="+2" color="blue">Additional results: diseases are grouped</font>

Author: <font color="blue">Guillem Escriba Molto</font>

E-mail: <font color="blue">guillem.escriba01@estudiant.upf.edu</font>

Date: <font color="blue">02/11/2021</font>

# 1. The diseasome bi-partite graph

In [1]:
import io
import csv
import pandas as pd

In [2]:


INPUT_FILENAME = "disease-genes.csv"
OUTPUT_DISEASOME_FILENAME = "diseasome.csv"

In [3]:

disease_genes = pd.read_csv(INPUT_FILENAME, sep=",")
disease_genes.set_index("ID", inplace=True)
disease_genes.head(10)

,Name,Genes,OMIM ID,Chromosome,Class
ID,,,,,
1,"17,20-lyase deficiency, isolated","CYP17A1, CYP17, P450C17",609300,10q24.3,Endocrine
1,"17-alpha-hydroxylase/17,20-lyase deficiency","CYP17A1, CYP17, P450C17",609300,10q24.3,Endocrine
3,2-methyl-3-hydroxybutyryl-CoA dehydrogenase de...,"HADH2, ERAB",300256,Xp11.2,Metabolic
4,2-methylbutyrylglycinuria,ACADSB,600301,10q25-q26,Metabolic
5,"3-beta-hydroxysteroid dehydrogenase, type II, ...",HSD3B2,201810,1p13.1,Metabolic
6,3-hydroxyacyl-CoA dehydrogenase deficiency,"HADHSC, SCHAD",601609,4q22-q26,Metabolic
7,3-Methylcrotonyl-CoA carboxylase 1 deficiency,"MCCC1, MCCA",609010,3q25-q27,Metabolic
7,3-Methylcrotonyl-CoA carboxylase 2 deficiency,"MCCC2, MCCB",609014,5q12-q13,Metabolic
8,"3-methylglutaconic aciduria, type I",AUH,600529,Chr.9,Metabolic


In [4]:
# Este código compacto se encarga de seleccionar las columnas deseadas y crear el dataframe de diseasome
diseasome = pd.DataFrame(list(zip(disease_genes.Name,disease_genes.Class, disease_genes.Genes)), columns = ["disorder","class","gene_list"])

In [5]:
# A partir de un dataframe este código se encarga de crear un archivo csv separado con tabulaciones "\t"
pd.DataFrame.to_csv(diseasome,OUTPUT_DISEASOME_FILENAME,'\t')

![Diseasome graph](diseasome.png)

<font size="+1" color="black">
    En este grafo observamos el mayor componente conexo del grafo diseasome, en dicho grafo se enlazam distintas enfermedades a los genes con los que guardan relación, ya sea por ser estos la causa de las enfermedades o por otros motivos. El grafo representado está completamente conectado al ser un CC del original. Este CC posee 194 nodos y 196 enlaces, su porcentaje de nodos respecto alos nodos totales de diseasome es: Nc/N = 194/4342 = 4'47%. Se puede apreciar que el tipo de enferedades en dicho CC es Cancer, tiene su sentido por la gran variedad que hay. El motivo por el que varias enfermedades del mismo tipo estén próximas se debe al tipo de gen que las causa, por ejemplo, las enfermedades de la vista estarán relacionadas con los genes que intevengan en la visión.</font>

# 2. The disease-disease graph

In [6]:
def intersection(list1, list2):            
    return(list(set(list1) & set(list2)))

OUTPUT_DISEASEDISEASE_FILENAME = "disease-disease.csv"

with io.open(OUTPUT_DISEASEDISEASE_FILENAME, "w") as fileout:   # Crea un archivo csv a partir de columnas de datos
    diseases = csv.writer(fileout, delimiter='\t', quotechar='"')  # csv con tabulaciones
    diseases.writerow(["disorder1","disorder2","ngenes1","ngenes2","class1","class2","ncommon_genes"]) # Nombres columnas
    for idx1, disorder1 in diseasome.iterrows(): # Iterar sobre la primera enfermedad a comparar genes
        gene_list_1 = disorder1["gene_list"].split(",") # Crear una lista de genes a partir de un string separado con comas
        for idx2, disorder2 in diseasome.iterrows(): # Iterar sobre la segunda enfermedad a comparar genes
            if disorder2["disorder"] != disorder1["disorder"]: # Compara que ambas enfermedades sean diferentes 
                gene_list_2 = disorder2["gene_list"].split(",") 
                common_genes = intersection(gene_list_1, gene_list_2) # Comprueba los genes comunes entre ambas listas
                if len(common_genes) > 0: # Si hay más de un gen común añade la fila al csv
                    diseases.writerow([disorder1["disorder"],disorder2["disorder"],len(gene_list_1),len(gene_list_2), disorder1["class"],disorder2["class"],len(common_genes)])





![Largest connected component of diseases graph](disease-disease-largest-cc.png)

![Diseases graph](disease-disease.png)

<font size="+1" color="black"> En estos grafos, siendo el principal el de abajo, disease-disease, tiene 1608 nodos y su mayor CC, el de arriba, tiene 132 CC lo que supone un Nc/N = 132/1608 = 8'21%. Podemos ver que a pesar de haberse reducido a más de la mitad el número de nodos del grafo, respecto diseasome, el número de nodos del mayor CC no se ha reducido proporcionalmente, mi hipótesis respecto a esto es que al tener una mayor conexión entre sí los elementos del CC era más probable que las enfermedades compartiesen genes entre sí. Otra cosa interesante que se puede obserbar en este grafo mejor que en el anterior es que las enfermedades del mismo tipo, en la gran mayoría de los casos están directamente relacionadas entre sí, lo que demuestra que son causadas por genes comunes. Los otros dos mayores CC, marcados en la segunda imagen tienen respectivamente, 25 y 18 nodos, un escaso 10-15% del mayor CC. Un hecho destacable es que los dos mayores CC tienen como tipo de enfermedad predominante el cancer, lo que puede dar a entender que son las enfermedades que más relación guardan entre sí.</font>

# 3. Additional results: diseases are grouped

In [7]:
def intersection(list1, list2):
    return(list(set(list1) & set(list2)))

OUTPUT_DISEASEDISEASE_FILENAME = "disease-disease-grouped.csv"

with io.open(OUTPUT_DISEASEDISEASE_FILENAME, "w") as fileout:
    diseases_grouped = csv.writer(fileout, delimiter='\t', quotechar='"')
    diseases_grouped.writerow(["disorder1","disorder2","ngenes1","ngenes2","class1","class2","ncommon_genes"])
    for idx1, disorder1 in diseasome.iterrows():
        gene_list_1 = disorder1["gene_list"].split(",")
        for idx2, disorder2 in diseasome.iterrows():
            if disorder2["disorder"] != disorder1["disorder"]:
                gene_list_2 = disorder2["gene_list"].split(",")
                common_genes = intersection(gene_list_1, gene_list_2)
                if len(common_genes) > 0: # Hasta aquí el funcionamiento es igual que en el anterior.
                    
                    disease_group1 = disorder1["disorder"].split(",") # Se dividen las enfermedades separadas con comas en una lista
                    disease_group2 = disorder2["disorder"].split(",")
                    
                    name1 = disease_group1[0] # Se selecciona como grupo de enfermedades el nombre de la primera Ej:"Epilepsy, progressive myoclonic 2B" pasa a ser Epilepsy 
                    if len(disease_group1[0])<3: # Para evitar casos como el 17,20-lyase deficiency, que se cortaria por 17
                        name1 = disease_group1[0]+','+disease_group1[1]
                    
                    name2 = disease_group2[0] # Lo mismo para el segundo grupo
                    if len(disease_group2[0])<3:
                        name2 = disease_group2[0]+','+disease_group2[1]
                        
                    diseases_grouped.writerow([name1, name2,len(gene_list_1),len(gene_list_2), disorder1["class"],disorder2["class"],len(common_genes)])
                

![Diseases grouped graph](disease-disease-grouped.png)

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>
